In [12]:
import os
for dirname, _, filenames in os.walk("dataset/nlp-getting-started/"):
    for filnames in filenames:
        print(os.path.join(dirname, filnames))

dataset/nlp-getting-started/test.csv
dataset/nlp-getting-started/train.csv
dataset/nlp-getting-started/sample_submission.csv


In [13]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import time

import torch
from torchtext import data
import torch.nn

In [14]:
test = pd.read_csv("./dataset/nlp-getting-started/test.csv")
train = pd.read_csv("./dataset/nlp-getting-started/train.csv")

# Data Pre Processing
텍스트 데이터 클리닝

In [15]:
train.shape

(7613, 5)

데이터 테이블 살펴보기

In [16]:
train.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


* label ==  1: 트위터에서 재난에 관한것
* label == 0: 트위터에서 재난이 아닌것

텍스트와 label열에만 관심이 있기 때문에 나머지는 삭제

In [17]:
train.drop(columns=["id", "keyword", "location"], inplace=True)

다음으로 분류 알고리즘이 관련 없는 정보와 혼동되지 않도록 텍스트를 클리닝하고 수정합니다.

In [18]:
def normalise_text(text):
    text = text.str.lower() # 모든 텍스트를 소문자로 변경합니다.
    text = text.str.replace(r"\#", "") # 해시태그 기호(#)를 제거합니다.
    text = text.str.replace(r"http\S+", "URL") # URL 주소를 'URL' 문자열로 대체합니다. \S+는 공백이 아닌 하나 이상의 문자에 해당합니다.
    text = text.str.replace(r"@", "") # '@' 기호를 제거합니다.
    text = text.str.replace(r"[^A-Za-z0-9()!?\'\`\"]", " ") # 알파벳, 숫자, 괄호, 느낌표, 물음표, 작은따옴표, 백틱, 쌍따옴표를 제외한 모든 문자를 공백으로 대체합니다.
    text = text.str.replace("\s{2,}", " ") # 두 개 이상의 연속된 공백을 하나의 공백으로 대체합니다.
    return text

In [19]:
train["text"] = normalise_text(train["text"])

클리닝한 데이터 확인

In [20]:
train["text"].head()

0    our deeds are the reason of this #earthquake m...
1               forest fire near la ronge sask. canada
2    all residents asked to 'shelter in place' are ...
3    13,000 people receive #wildfires evacuation or...
4    just got sent this photo from ruby #alaska as ...
Name: text, dtype: object

데이터 스플릿

In [21]:
train_df, valid_df = train_test_split(train)

In [22]:
train_df.head()

,text,target
5097,#japan #fukushima the most serious man made di...,1
7383,i like the weird ones like rain of mystical or...,0
5663,tell the united nations: plantations are not f...,0
6339,investigators say a fatal virgin galactic spac...,1
4013,who is bringing the tornadoes and floods. who ...,0


In [23]:
valid_df.head()

,text,target
4205,tomdean86 he's alright but hazard/willian ain'...,0
4428,who is tomislav salopek the islamic state's mo...,0
351,one direction is my pick for http://t.co/q2ebl...,0
2973,sometimesi can't even breathe well\ni feel lik...,0
7151,#sismo m 1.9 - 5km s of volcano hawaii: time20...,1


시드를 고정하여 결과 재현성을 높입니다.

In [24]:
SEED = 42

torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

텍스트 데이터를 처리하기 위해 Field 객체를 만들어야 합니다. 이러한 Field 객체에는 텍스트를 Tensor로 변환하기 위한 정보가 포함됩니다. 우리는 두 개의 파라미터를 설정할 것입니다.

tokenize = spacy 그리고 include_arguments=True 이것은 SpCy가 텍스트의 토큰화를 위해 사용된다는 것이고 Field객체가 텍스트의 길이를 포함해야 한다는 것을 의미합니다.

우리는 나중에 이러한 Field 객체를 사용하여 어휘(vocabulary)를 만들 것이고, 이것은 우리가 모든 토큰에 대한 숫자 표현을 만드는 데 도움이 될 것입니다.

LabelField는 필드 주변의 얕은 wrappper로 데이터 레이블에 유용합니다.

In [27]:
TEXT = data.Field(tokenize = 'spacy', include_lengths = True)
LABEL = data.LabelField(dtype = torch.float)

AttributeError: module 'torchtext.data' has no attribute 'Field'